# Datastory 2

In [9]:
import pandas as pd
import plotly.express as px

# Load the sustainable energy data
energy_df = pd.read_csv('/Users/delaram/infovis/docs/nuclear_data/global-data-on-sustainable-energy (1).csv')

# Calculate the percentage of nuclear electricity
energy_df['Nuclear_Electricity_Percentage'] = (energy_df['Electricity from nuclear (TWh)'] /
                                             (energy_df['Electricity from fossil fuels (TWh)'] +
                                              energy_df['Electricity from nuclear (TWh)'] +
                                              energy_df['Electricity from renewables (TWh)'])) * 100

# Replace NaN values with 0 for better visualization
energy_df['Nuclear_Electricity_Percentage'] = energy_df['Nuclear_Electricity_Percentage'].fillna(0)

# Load the nuclear incidents data
incidents_df = pd.read_csv("/Users/delaram/infovis/docs/nuclear_data/Nuclear Incidents - Sheet3.csv")

# Clean and prepare incidents data
incidents_df.columns = incidents_df.columns.str.strip()
incidents_df.columns.values[6] = "INES level"

incidents_df['Numbers of Direct Deaths'] = incidents_df['Numbers of Direct Deaths'].replace('', pd.NA).fillna('Unknown')
incidents_df['Numbers of InDirect Deaths'] = incidents_df['Numbers of InDirect Deaths'].replace('', pd.NA).fillna('Unknown')
incidents_df['INES level'] = incidents_df['INES level'].replace('', pd.NA).fillna('Unknown')

incidents_df['hover_text'] = (
    "<b>Incident:</b> " + incidents_df['Incident'] + "<br>" +
    "<b>Location:</b> " + incidents_df['Location'] + "<br>" +
    "<b>Date:</b> " + incidents_df['Date'] + "<br>" +
    "<b>Direct Deaths:</b> " + incidents_df['Numbers of Direct Deaths'].astype(str) + "<br>" +
    "<b>Indirect Deaths:</b> " + incidents_df['Numbers of InDirect Deaths'].astype(str) + "<br>" +
    "<b>Category:</b> " + incidents_df['Category'] + "<br>" +
    "<b>INES Level:</b> " + incidents_df['INES level'].astype(str) + "<br>" +
    "<b>Latitude:</b> " + incidents_df['Latitude'].astype(str) + "<br>" +
    "<b>Longitude:</b> " + incidents_df['Longitude'].astype(str)
)

# Create the base map with nuclear energy percentage
fig = px.choropleth(energy_df,
                    locations="Entity",
                    locationmode="country names",
                    color="Nuclear_Electricity_Percentage",
                    hover_name="Entity",
                    hover_data=["Year", "Electricity from nuclear (TWh)"],
                    animation_frame="Year",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    range_color=[0, 100],
                    title="Nuclear Energy Use and Incidents Worldwide",
                    labels={'Nuclear_Electricity_Percentage':'Nuclear Electricity (%)'},
                    height=600)

# Add nuclear incidents as scatter points
fig.add_trace(
    px.scatter_geo(incidents_df,
                   lat='Latitude',
                   lon='Longitude',
                   hover_name='Incident',
                   hover_data={'Latitude': False, 'Longitude': False},
                   custom_data=['hover_text'],
                   color_discrete_sequence=['yellow']).data[0]  # Kleur veranderd naar geel
)

# Update layout and traces
fig.update_traces(
    selector=dict(type='choropleth'),
    marker_line_width=0.5,
    marker_line_color='white'
)

fig.update_traces(
    selector=dict(type='scattergeo'),
    hovertemplate="%{customdata[0]}<extra></extra>",
    marker=dict(size=10, symbol='circle', color='yellow')  # Nu een gele cirkel
)

fig.update_layout(
    margin={"r": 0, "t": 40, "l": 0, "b": 0},
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='natural earth'
    ),
    coloraxis_colorbar=dict(
        title="Nuclear %",
        thickness=15,
        len=0.75
    ),
    sliders=[{
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Year:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 300, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0
    }]
)

# Adjust animation settings
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 500

fig.show()